In [1]:
import cpp_soft_info

# Load Data

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from result_saver import SaverProvider
provider = SaverProvider()

In [8]:
import numpy as np
from Scratch import metadata_loader

DEVICE = 'ibm_sherbrooke'

md = metadata_loader(_extract=True, _drop_inutile=False)
# print(md.head())
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["notebook_name"] == "bigger_rep_codes"]
max_distance = int(max(md.distance))
max_distance = 30
md = md[md["distance"] == max_distance]
md = md[md["backend_name"]==DEVICE]


md = md[md["meas_level"] == 1]
# md = md[:2]

d = 30
synd_rounds = 30

md

,creation_date,notebook_name,backend_name,job_id,job_name,job_metadata,tags,meas_level,init_qubits,meas_return,...,job_status,execution_date,extra,optimization_level,code,distance,rounds,logical,layout,descr
258,2023-10-30 09:45:17.340386+01:00,bigger_rep_codes,ibm_sherbrooke,cmzpt78vcq70008qf1ag,NaN,{},[],1.0,True,single,...,JobStatus.DONE,2023-10-31 17:07:46.121619+01:00,NaN,NaN,RepetitionCodeCircuit,30.0,30,0,_is_hex=True,Run bigger Repetition codes v4: new distances ...
76,2023-10-29 14:47:58.814875+01:00,bigger_rep_codes,ibm_sherbrooke,cmz653m3r3vg008wf9j0,NaN,{},[],1.0,True,single,...,JobStatus.DONE,2023-10-29 15:12:44.092651+01:00,NaN,NaN,RepetitionCodeCircuit,30.0,30,1,_is_hex=True,Run bigger Repetition codes
75,2023-10-29 14:47:43.903639+01:00,bigger_rep_codes,ibm_sherbrooke,cmz64zvvcq70008qdxcg,NaN,{},[],1.0,True,single,...,JobStatus.DONE,2023-10-29 15:11:38.373711+01:00,NaN,NaN,RepetitionCodeCircuit,30.0,30,0,_is_hex=True,Run bigger Repetition codes


In [5]:
memories = {}
jobs = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()
    jobs[mmr_name] = job_id

memory_str = "mmr_log_0"
memory = memories[memory_str][:]
job_id = jobs[memory_str]
print(memory.shape)

(1111, 900)


# Initialize the code

In [6]:
import stim
import pymatching

from soft_info import get_repcode_layout, get_repcode_IQ_map
from Scratch import create_or_load_kde_grid


layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend(DEVICE), _is_hex=True)
qubit_mapping = get_repcode_IQ_map(layout, synd_rounds) #Hardcoded for repetition codes
print("layout:", layout)
print("len(layout):", len(layout), " = distance * code qubits + (distance-1) * link qubits")

layout: [114, 116, 118, 120, 122, 104, 102, 100, 98, 96, 94, 75, 77, 79, 81, 83, 85, 66, 64, 62, 60, 58, 56, 37, 39, 41, 43, 45, 47, 113, 115, 117, 119, 121, 111, 103, 101, 99, 97, 95, 90, 76, 78, 80, 82, 84, 73, 65, 63, 61, 59, 57, 52, 38, 40, 42, 44, 46, 35]
len(layout): 59  = distance * code qubits + (distance-1) * link qubits


In [7]:
other_date = None # if none then it will find the closest to the tobecalib_job date
# other_date = "2023-11-10T10:30:00" # "2023-11-22T" works to
grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, job_id, 300, 2, other_date=other_date)

# takes 10s

Specified job execution date: 2023-10-31 17:07:46.121619+01:00
Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00. Retrieving kde grid...
Creation dates: 2023-10-27 07:46:44.189709+00:00 and 2023-10-27 07:46:41.326236+00:00
Execution dates: 2023-10-27 08:32:22.841567+00:00 and 2023-10-27 08:32:04.977564+00:00
Searching for ibm_sherbrooke and 23.10.27_07h46_300pts_2std


#### Noise model stim

In [9]:
from datetime import datetime

def get_avgs_from_dict(noise_dict):
    '''
    Takes noise dictionary as an argument then returns length 5
    array defining average noise levels: [init, idle, RO, single-, two-qubit gate]
    '''
    init_avg = np.average([noise_dict[qubit]['init'] for qubit in noise_dict])
    idle_avg = np.average([noise_dict[qubit]['idle'] for qubit in noise_dict])
    RO_avg = np.average([noise_dict[qubit]['RO'] for qubit in noise_dict])
    single_avg = 1e-4#np.average([noise_dict[qubit]['init'] for qubit in noise_dict])
    two_gate_avg = np.average([noise_dict[qubit]['2-gate'][connection]
                        for qubit in noise_dict
                        for connection in noise_dict[qubit]['2-gate']
                        if connection != 'default'])
    return [init_avg, idle_avg, RO_avg, single_avg, two_gate_avg]
# End get_avgs_from_dict


def get_noise_dict_from_backend(backend, layout, date=None):
    '''
    Takes a given backend and a mapping object and returns a noise dictionary
    to pass to a new Cross_Platform_Code/Hex_Code to have accurate backend
    specific noise information based on calibration data.
    '''
    # Pre-reqs
    noise_dict = {}
    round_time = 1000e-9
    if date is None:
        t = None
    else:
        t = datetime(day=int(date[-2:]), month=int(date[-4:-2]), year=int(date[:4]))
    properties = backend.properties(datetime=t)
    # Initializing each qubit with single qubit noise
    for qubit in layout:
        # Defining ROI error
        ROI_error = properties.readout_error(qubit)
        # Defining idle error
        t1 = properties.t1(qubit)
        t2 = properties.t2(qubit)
        t_time = min(t1, t2)
        idle_error = 1 - np.exp(-round_time / t_time)
        # Updating dictionary
        noise_dict[qubit] = {}
        noise_dict[qubit]['init'] = ROI_error
        noise_dict[qubit]['idle'] = idle_error
        noise_dict[qubit]['RO'] = ROI_error
        noise_dict[qubit]['gate'] = 1e-4
        noise_dict[qubit]['2-gate'] = {'default': 1e-2}
    # Two qubit gate noise
    all_qiskit_indexes = layout 
    for pair in backend.coupling_map:
        # Making sure connection is in circuit
        if pair[0] in all_qiskit_indexes and pair[1] in all_qiskit_indexes:
            # Getting two qubit gate error
            two_gate_error = properties.gate_error('ecr', pair)
            if two_gate_error > .5:
                two_gate_error = .5
            # # Updating dictionary
            # stim_index_0 = map.get_stim_index(qiskit_index=pair[0])
            # stim_index_1 = map.get_stim_index(qiskit_index=pair[1])
            noise_dict[pair[0]]['2-gate'][pair[1]] = two_gate_error
            noise_dict[pair[1]]['2-gate'][pair[0]] = two_gate_error
    return noise_dict
# End get_noise_dict_from_backend

In [10]:

date = "2023-10-30 09:45:17.340386+01:00" #md.creation_date[0] # md[0] for log 1
noise_dict = get_noise_dict_from_backend(provider.get_backend(DEVICE), layout, date=None) # takes noise closest to today
print("noise_dict:", noise_dict)
avgs_noise = get_avgs_from_dict(noise_dict)
print("avgs_noise [init_avg, idle_avg, RO_avg, single_avg, two_gate_avg]:", avgs_noise)

# takes 4s

noise_dict: {114: {'init': 0.012799999999999923, 'idle': 0.0036593906155806266, 'RO': 0.012799999999999923, 'gate': 0.0001, '2-gate': {'default': 0.01, 113: 0.005452106269309642, 115: 0.009751021836470197}}, 116: {'init': 0.00770000000000004, 'idle': 0.005675580168901018, 'RO': 0.00770000000000004, 'gate': 0.0001, '2-gate': {'default': 0.01, 115: 0.006665310149900822, 117: 0.01269433701778852}}, 118: {'init': 0.0040000000000000036, 'idle': 0.004544406240744481, 'RO': 0.0040000000000000036, 'gate': 0.0001, '2-gate': {'default': 0.01, 117: 0.005921658318412182, 119: 0.010853065924460586}}, 120: {'init': 0.010000000000000009, 'idle': 0.00352197246803454, 'RO': 0.010000000000000009, 'gate': 0.0001, '2-gate': {'default': 0.01, 119: 0.006469636251456773, 121: 0.005020561482665692}}, 122: {'init': 0.010199999999999987, 'idle': 0.0037506010734535966, 'RO': 0.010199999999999987, 'gate': 0.0001, '2-gate': {'default': 0.01, 121: 0.007181297829149641, 111: 0.010637920476153251}}, 104: {'init': 0.0

In [11]:
from soft_info import draw_matching_graph



circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=d,
                                rounds=synd_rounds,
                                after_clifford_depolarization= avgs_noise[4], #two-qubit-fidelity,
                                after_reset_flip_probability= 0, #reset error,
                                before_measure_flip_probability= avgs_noise[2], #measurement error,
                                before_round_data_depolarization= 0)#avgs_noise[1]) #idle error)
# print(circuit)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)
# draw_matching_graph(matching, d, synd_rounds, figsize=(20, 20))   
# takes 10s

# Decode

In [12]:
counts = cpp_soft_info.get_counts(memory[:], qubit_mapping, grid_dict, processed_scaler_dict, synd_rounds)

In [13]:
matching = pymatching.Matching.from_detector_error_model(model)
# cpp_soft_info.reweight_edges_based_on_error_probs(matching._matching_graph, counts, False, "spitz")

p_data = 6.869e-3 # mean sherbrooke noise
num_errors = cpp_soft_info.decode_IQ_shots(matching._matching_graph, memory,
                                           synd_rounds, qubit_mapping, grid_dict,
                                           processed_scaler_dict, p_data=-1, p_mixed=-1, #p_mixed=1e-80, for d=30 
                                           common_measure=-1, _bimodal=False, merge_strategy = "replace")
print("num_errors:", num_errors, "out of", len(memory), "shots")
 
# takes 5s

num_errors: 15 out of 1111 shots


In [14]:
matching = pymatching.Matching.from_detector_error_model(model)
num_errors = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, memory,
                                           synd_rounds, qubit_mapping, grid_dict,
                                           processed_scaler_dict)
print("num_errors:", num_errors, "out of", len(memory), "shots")

# takes 1s

num_errors: 46 out of 1111 shots


In [17]:
matching = pymatching.Matching.from_detector_error_model(model)

def weight_to_prob(weight):
    return 1/(1+np.exp(weight))

p_data = 6.869e-3 # mean sherbrooke ECR error
p_mixed = p_data/1 # Same as weighted
# p_meas = 1e-3
p_meas = 15.900e-2 # random found number


p_data = -1
p_mixed = -1
p_meas = -1

num_errors = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, memory, 
                                           synd_rounds, qubit_mapping, grid_dict, processed_scaler_dict,
                                           p_data, p_mixed, p_meas, common_measure=-1)

print("num_errors:", num_errors, "out of", len(memory), "shots")
         
# takes 1s

num_errors: 21 out of 1111 shots
